### System rekomendujący na bazie Netflix Prize award

1. Zacznijmy od obejrzenia wykładu [rozdział 9](http://www.mmds.org/)
2. proszę ściągnac bazę netflixa o której była mowa w/w wykładzie [z kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) (2GB) 

Zaczniemy od wczytania danych, ponizszy listing wczytuje dane z jednego pliku i robi z nich trójkę (user, product, rating). Wykorzystamy do tego predefiniowany obiekt Rating z mlib.recommendation (prosze zwrócic uwagę że w tej konwencji nasz film będzie produktem)

#### Zadanie 1:
zmodyfikuj poniższy listing tak aby wczytywać wsystkie pięc plików

In [1]:
import pyspark
from pyspark.mllib.recommendation import Rating
from pyspark import SparkContext

# tutaj zaincjalizujemy klaster (na jednym komputerze) sparka
# zmodyfikujemy nieco std ustawienia maszyny java zwiększając domyslne (bardzo małe) limity pamieci
# local[*] oznacza że użyjemy wsystkich rdzeni - jeśli zabrankie nam ramu możemy zmniejszyć ilość tą ilość
# polecam spoglądać do konsoli na http://localhost:4040/ aby monitorować zużycie zasobów

conf = pyspark.SparkConf().setAppName("recommendation")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '20G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

#files = ['./RS/combined_data_1.txt',
        #'./RS/combined_data_2.txt',
        #'./RS/combined_data_3.txt',
        #'./RS/combined_data_4.txt'] # Ufam, że użytkownik wprowadził nazwy plików, które istnieją...
files = [ './RS/combined_data_1.txt' ]

ratings = []
for single_name in files:
    f = open(single_name)
    for i, line in enumerate(f):
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line[:-1])        
        else:
            user_id, rating, _ = line.split(',')
            r = Rating(int(user_id), int(movie_id), int(rating))
            ratings.append(r)
    f.close()
    print("File '" + single_name + "' loaded!")

print('Finished! Processed lines: ' + str(len(ratings)))

File './RS/combined_data_1.txt' loaded!
Finished! Processed lines: 24053764


In [2]:
#tutaj wczytamy identyfikatory filmów i ich tytuły

f = open('./RS/movie_titles.csv', encoding = "ISO-8859-1")
g = [l.strip().split(',') for l in f.readlines()]
id2title = {int(a[0]):','.join(a[2:]) for a in g}
f.close()
print('Finished!')

Finished!


In [3]:
id2title
print('Finished!')

Finished!


In [4]:
import time

# utworzmy z naszej listy ratingow zasob rdd (rozproszenie)
start = time.time()
ratings_rdd = sc.parallelize(ratings)
end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Finished! Elapsed time: 32.11 seconds.


#### Zadanie 2
Przefiltruj ratings_rdd aby wziac pod uwage filmy ktory maja co najmniej 50 ocen
tip: zrób napierw liste par (movie_id, Rating) i pogrupuj ją przy użyciu GroupByKey a nstępnie przefiltruj

In [ ]:
ratings_pairs = ratings_rdd.map(lambda r: (r.product, r))
rg = ratings_pairs.groupByKey() # ???

# ???
ratings_filtered = rg.filter(lambda r: len(r[1]) >= 50).collect() # ???


# zasob rdd na przefiltrowanym obiekcie
ratings_filtered = sc.parallelize(ratings_filtered)

In [6]:
# TROCHĘ INNYM SPOSOBEM - na około (zmiana na DataFrame po to by korzystać z SQL Queries)
import pyspark.sql.functions as ps_functions
from pyspark.sql import SQLContext
from pyspark.sql import Window
import time

start = time.time()
data = SQLContext(sc)
print('Stage 1 - finished!')

all_ratings = data.createDataFrame(ratings_rdd)
print('Stage 2 - finished!')

fragment = Window.partitionBy('product')
print('Stage 3 - finished!')

query = all_ratings.select('user', 'product', 'rating', ps_functions.count('product').over(fragment).alias("ratings_count"))
print('Stage 4 - finished!')

marker = query.filter("ratings_count >= 50").select('user','product','rating')
print('Stage 5 - finished!')

ratings = sc.parallelize(marker.collect())
print('Stage 6 - finished!')

end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')
# TROCHĘ INNYM SPOSOBEM - na około (zmiana na DataFrame po to by korzystać z SQL Queries)

Stage 1 - finished!
Stage 2 - finished!
Stage 3 - finished!
Stage 4 - finished!
Stage 5 - finished!
Stage 6 - finished!
Finished! Elapsed time: 161.56 seconds.


Dokonamy teraz faktoryzacji macierzy, nasej utility Matrix (zasób RDD ratings_filtered jest własnie taką macierzą) przy użyciu aproksymacji algorytmu spadku gradientowego
![alt](https://edersoncorbari.github.io/assets/images/blog/als-matrix-rec-calc.png)

In [7]:
from pyspark.mllib.recommendation import ALS
import time

rank = 10
numIterations = 10

start = time.time()
model = ALS.train(ratings, rank, numIterations)
end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Finished! Elapsed time: 81.59 seconds.


Zrzucilismy uzytkowników i filmy na nowy wymiar o wielkości 10. Możemy wprost poprosić o te macierze

In [8]:
users  = model.userFeatures()
movies = model.productFeatures()

Gęsta ale 'wąska' macierz movies zmieści się nam już bezproblemowo w RAM, a zatem zróbmy z niej po prostu macierz numpy

In [9]:
import numpy as np

movies_mtx = np.array(movies.map(lambda rv:rv[1]).collect())
movie2row_number = movies.map(lambda rv:rv[0]).collect() #movie id to row number

Zróbmy prosty ekesperyment, zobaczmy czy jakie filmy są 'podobne' do 'Władcy pierścieni'

In [10]:
id2title[1757] # 'The Lord of the Rings'

lotr_idx = movie2row_number.index(1757)
lotr_vector = movies_mtx[lotr_idx]
print('Title: ' + id2title[1757])
print(lotr_vector) #to jest wladca pierscienie rzutowany na latent space

Title: The Lord of the Rings
[-0.40519634  0.3050507  -0.53807741 -0.01964168  0.31152999 -0.15673974
 -0.26772276  0.13672216 -0.47071424  0.00552696]


In [11]:
# obliczmy macierz odleglosci 
from scipy.spatial import distance

ds = distance.cdist([lotr_vector], movies_mtx, 'cosine')[0]
dist = ds.argsort()[:10] # 10 - liczba filmów

print('Distance: ' + str(dist))

for i in dist:
    print('Movie title using ID ' + str(i) + ': ' + id2title[movie2row_number[i]])

Distance: [1561 3816 4170 1279 2782  142 2501 2413 4126  824]
Movie title using ID 1561: The Lord of the Rings
Movie title using ID 3816: Alien: Resurrection: Collector's Edition
Movie title using ID 4170: Campfire Stories
Movie title using ID 1279: Firestarter
Movie title using ID 2782: The Return of the King
Movie title using ID 142: The Masque of the Red Death / The Premature Burial
Movie title using ID 2501: Silver Bullet
Movie title using ID 2413: Hellbound: Hellraiser II
Movie title using ID 4126: Lord of Illusions
Movie title using ID 824: Taste the Blood of Dracula


In [12]:
for i in dist:
    print('Distance value: ' + str(ds[i]) + " refers movie: " + id2title[movie2row_number[i]])

# print(str(ds[2018]) + ': ' + id2title[movie2row_number[2018]])
# print(str(ds[928]) + ': ' + id2title[movie2row_number[928]])

Distance value: 0.0 refers movie: The Lord of the Rings
Distance value: 0.06194802950162004 refers movie: Alien: Resurrection: Collector's Edition
Distance value: 0.06650902880989118 refers movie: Campfire Stories
Distance value: 0.07866967773630595 refers movie: Firestarter
Distance value: 0.08017573924912091 refers movie: The Return of the King
Distance value: 0.08197936900477432 refers movie: The Masque of the Red Death / The Premature Burial
Distance value: 0.08375122220501197 refers movie: Silver Bullet
Distance value: 0.08576027762557836 refers movie: Hellbound: Hellraiser II
Distance value: 0.0861341589001966 refers movie: Lord of Illusions
Distance value: 0.08635267782427059 refers movie: Taste the Blood of Dracula


Odległosć 0.0 to oczywiscie ten sam obiekt a zatem interesuje nas drugi w kolejności wpis. I cóż za niespodzianka? ludzie którzy ocenili wysoko Władce pierścieni ocenili również wysokos Powrót Króla

Na ile cech powinniśmy faktoryzować Utility Matrix? 

#### Zadanie 3
Dobierz paramter rank (ilość cech) dla obiektu ALS dzieląc zbiór ratings_filtered na 80% vs 20% (treningowy i testowy, tak jak na wykłdzie). Następnie wykorzystać metodę ALS.recommendProductsForUsers aby polecić filmy i zweryfikuj ile filmów udało Ci sie prawidłowo polecić ze zbioru testowego. Odpowiednio modyfikuj parametr rank aby otrzymać względnie wysoki wynik rekomendacji jednocześnie utrymując w miare wąskie macierze

In [13]:
#files = ['./RS/combined_data_1.txt',
        #'./RS/combined_data_2.txt',
        #'./RS/combined_data_3.txt',
        #'./RS/combined_data_4.txt'] # Ufam, że użytkownik wprowadził nazwy plików, które istnieją...
files = [ './RS/combined_data_1.txt' ]

ratings = []
for single_name in files:
    f = open(single_name)
    for i, line in enumerate(f):
        line = line.strip()
        if line.endswith(':'):
            movie_id = int(line[:-1])        
        else:
            user_id, rating, _ = line.split(',')
            r = Rating(int(user_id), int(movie_id), int(rating))
            ratings.append(r)
    f.close()
    print("File '" + single_name + "' loaded!")

print('Loading finished! Processed lines: ' + str(len(ratings)))

f = open('./RS/movie_titles.csv', encoding = "ISO-8859-1")
g = [l.strip().split(',') for l in f.readlines()]
id2title = {int(a[0]):','.join(a[2:]) for a in g}
f.close()
print('IDs and titles loaded!!')

start = time.time()
ratings_rdd = sc.parallelize(ratings)
end = time.time()
print('RDD resource created! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

File './RS/combined_data_1.txt' loaded!
Loading finished! Processed lines: 24053764
IDs and titles loaded!!
RDD resource created! Elapsed time: 31.39 seconds.


In [14]:
import pyspark.sql.functions as ps_functions
from pyspark.sql import SQLContext
from pyspark.sql import Window
import time

start = time.time()
data = SQLContext(sc)
print('Stage 1 - finished!')

all_ratings = data.createDataFrame(ratings_rdd) # Tu zawsze pojawia się error przy pierwszym uruchomieniu. Drugie uruchomienie już go nie wywołuje
print('Stage 2 - finished!')

fragment = Window.partitionBy('product')
print('Stage 3 - finished!')

query = all_ratings.select('user', 'product', 'rating', ps_functions.count('product').over(fragment).alias("ratings_count"))
print('Stage 4 - finished!')

marker = query.filter("ratings_count >= 50").select('user','product','rating')
print('Stage 5 - finished!')

ratings = sc.parallelize(marker.collect())
print('Stage 6 - finished!')

end = time.time()
print('Finished! Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

Stage 1 - finished!
Stage 2 - finished!
Stage 3 - finished!
Stage 4 - finished!
Stage 5 - finished!
Stage 6 - finished!
Finished! Elapsed time: 191.64 seconds.


In [15]:
from pyspark.mllib.recommendation import ALS
import numpy as np
import math
import time

############################################################

train, valid, test = ratings.randomSplit([6,2,2], seed=0)
emptyTest = test.map(lambda r: (r[0], r[1]))
emptyValid = valid.map(lambda r: (r[0], r[1]))
print('Finished!')

Finished!


In [16]:
ranks = range(1, 21)

errorsList = []
models = []
for i in range(len(ranks)):
    errorsList.append(0)
ind = 0

remembered_error = float('inf')
the_best_rank = -1
for rank in ranks:
    try: # POJAWIA SIĘ PROBLEM Z TCP OBSŁUGIWANYM PRZEZ JAVĘ - dlatego try-except
        start = time.time()
        # Jeżeli wystąpi `error` to w bloku poniżej
        model = ALS.train(train, rank, seed=5, iterations=20, lambda_=0.1)
        pred = model.predictAll(emptyValid).map(lambda r: ((r[0], r[1]), r[2]))
        rates = valid.map(lambda r: ((r[0], r[1]), r[2])).join(pred)
        newError = math.sqrt(rates.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        # Jeżeli wystąpi `error` to w bloku powyżej
        errorsList[ind] = newError
        ind += 1
        if newError < remembered_error:
            remembered_error = newError
            the_best_rank = rank
        end = time.time()
        # Zapamiętaj ostatni poprawnie wykonany zbiór zmiennych (poniżej)
        last_correct_model = model
        last_correct_pred = pred
        last_correct_rates = rates
        last_correct_newError = newError
        # Zapamiętaj ostatni poprawnie wykonany zbiór zmiennych (powyżej)
        print('Training completed for rank: ' + str(rank) + '/' + str(len(ranks)) + '. Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')
    except:
        # W razie błędu odzyskaj ostatni poprawnie wykonany zbiór zmiennych (poniżej)
        model = last_correct_model
        pred = last_correct_pred
        rates = last_correct_rates
        newError = last_correct_newError
        # W razie błędu odzyskaj ostatni poprawnie wykonany zbiór zmiennych (powyżej)
        print('Rank ' + str(rank) + ' has encountered a problem! Data restored.')
print("The best rank %s" % the_best_rank)

Training completed for rank: 1/20. Elapsed time: 171.41 seconds.
Training completed for rank: 2/20. Elapsed time: 170.07 seconds.
Training completed for rank: 3/20. Elapsed time: 176.82 seconds.
Training completed for rank: 4/20. Elapsed time: 167.7 seconds.
Training completed for rank: 5/20. Elapsed time: 171.93 seconds.
Training completed for rank: 6/20. Elapsed time: 181.9 seconds.
Rank 7 has encountered a problem! Data restored.
Training completed for rank: 8/20. Elapsed time: 204.33 seconds.
Training completed for rank: 9/20. Elapsed time: 196.27 seconds.
Training completed for rank: 10/20. Elapsed time: 199.04 seconds.
Training completed for rank: 11/20. Elapsed time: 212.8 seconds.
Rank 12 has encountered a problem! Data restored.
Training completed for rank: 13/20. Elapsed time: 208.94 seconds.
Training completed for rank: 14/20. Elapsed time: 230.77 seconds.
Training completed for rank: 15/20. Elapsed time: 221.64 seconds.
Rank 16 has encountered a problem! Data restored.
Trai

In [17]:
train, test = ratings.randomSplit([8,2], seed=0)
emptyTest = test.map(lambda r: (r[0], r[1]))

############################################################

start = time.time()
model = ALS.train(train, the_best_rank, seed=5, iterations=20, lambda_=0.1)
pred = model.predictAll(emptyTest).map(lambda r: ((r[0], r[1]), r[2]))
rates = test.map(lambda r: ((r[0], r[1]), r[2])).join(pred)
end = time.time()
print('Test completed for rank: ' + str(the_best_rank) + ' Elapsed time: ' + str(round(end - start, 2)) + ' seconds.')

############################################################

products = model.recommendProductsForUsers(10)
products.first()

Test completed for rank: 20 Elapsed time: 196.59 seconds.


(1374416,
 (Rating(user=1374416, product=2700, rating=4.866043521149377),
  Rating(user=1374416, product=3033, rating=4.721042942324603),
  Rating(user=1374416, product=1230, rating=4.708888030404897),
  Rating(user=1374416, product=2102, rating=4.6854508074529715),
  Rating(user=1374416, product=3456, rating=4.6836380275702485),
  Rating(user=1374416, product=223, rating=4.674958529953608),
  Rating(user=1374416, product=3444, rating=4.671594230892056),
  Rating(user=1374416, product=724, rating=4.635454454397707),
  Rating(user=1374416, product=1418, rating=4.603810422278698),
  Rating(user=1374416, product=2172, rating=4.603664821979537)))

In [18]:
import numpy as np
import math

productions = model.productFeatures()
productions_matrix = np.array(movies.map(lambda rv:rv[1]).collect())
m2rn = productions.map(lambda rv:rv[0]).collect()

############################################################

print('Recommended movies: ')
for i in enumerate(products.first()[1]):
    product_index = i[1][1]
    print("\"" + id2title[m2rn.index(product_index)] + "\" for value %s" % i[1][2])

id2title[1757]
lotr_idx = m2rn.index(1757)
lotr_vector = productions_matrix[lotr_idx]

############################################################

print('\nTitle of watched movie: ' + id2title[1757])
print('Vector of watched movie:')
print(lotr_vector)

Recommended movies: 
"Fight Club: Bonus Material" for value 4.866043521149377
"Thomas & Sarah" for value 4.721042942324603
"The Element of Crime" for value 4.708888030404897
"Niea 7" for value 4.6854508074529715
"Tuck Everlasting" for value 4.6836380275702485
"Cinderella II" for value 4.674958529953608
"Creepshow" for value 4.671594230892056
"Scratch" for value 4.635454454397707
"Black Orpheus" for value 4.603810422278698
"King Cobra" for value 4.603664821979537

Title of watched movie: The Lord of the Rings
Vector of watched movie:
[-0.40519634  0.3050507  -0.53807741 -0.01964168  0.31152999 -0.15673974
 -0.26772276  0.13672216 -0.47071424  0.00552696]


#### Zadanie 4
Zrób program "co obejrzeć dziś wieczorem".
Program powinien pokazaywać 5 tytułow, uzytkownik wskazuje jeden lub kilka z nich i system pokazuje kolejne rekomendacje. Każda kolejna rekomendacja powinna wskazywać coraz lepsze rekomendacje. 

problem można rozwiazać na kilka sposób. Można pogrupować przestrzeń filmów (macierz P.T) i pokazywać z różnych np. centroidy klastrów (które będą reprezentować gatunki filmów (prawdopodobnie) ). 

każde kolejne wybranie filmu będzie wymagało stworzenie nowego sztuznego użytkownika z ratingiem. W każdej iteracji takiego użytkownika należy rzucic na macierz Q (czyli na latentną macierz użytkownika). Przypatrz się dokładnie sposobowi mnożenia na obrazku powyżej i zastanów się jak otrzymać takiego użytkownika w przestrzeni cech?). Przypominam że macierz P.T jest nieodracalna ale można wykorzystać pseudo odwrotnosć.